In [ ]:
# %% [markdown]
"""
# Restaurant Sales Data Analysis

This notebook analyzes sales data from a restaurant chain across multiple cities, including:
- Sales trends and patterns
- Product performance
- Manager performance
- Payment method preferences
- Monthly revenue analysis
"""

# %%
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configure display settings
pd.set_option('display.float_format', '{:,.2f}'.format)
plt.style.use('seaborn')
sns.set_palette('pastel')

# %% [markdown]
"""
## Data Loading and Cleaning
"""
# %%
def load_and_clean_data(filepath):
    """
    Load and clean the sales dataset
    Returns cleaned DataFrame
    """
    try:
        # Load data and handle header row
        df = pd.read_excel(filepath)
        df.columns = df.iloc[0]  # Set first row as column names
        df = df.drop(0).reset_index(drop=True)
        
        # Clean column names and drop unwanted columns
        df.columns = [col.strip().replace(' ', '_') for col in df.columns]
        if 'Unnamed:_0' in df.columns:
            df = df.drop(columns='Unnamed:_0')
            
        # Clean manager names
        df['Manager'] = df['Manager'].str.strip().str.replace(r'\s+', ' ', regex=True)
        
        # Remove duplicates
        initial_count = len(df)
        df = df.drop_duplicates()
        print(f"Removed {initial_count - len(df)} duplicate records")
        
        # Handle specific duplicate Order IDs
        duplicate_orders = df[df['Order_ID'].duplicated(keep=False)]
        if not duplicate_orders.empty:
            print(f"Found {len(duplicate_orders)} potentially duplicate orders")
            # In practice, would investigate these further before dropping
        
        # Convert data types
        df['Quantity'] = df['Quantity'].astype(float).round().astype(int)
        df['Price'] = df['Price'].astype(float)
        df['Order_ID'] = df['Order_ID'].astype(int)
        df['Date'] = pd.to_datetime(df['Date'])
        
        # Calculate revenue
        df['Revenue'] = df['Price'] * df['Quantity']
        
        # Extract month
        df['Month'] = df['Date'].dt.month
        
        return df
    
    except Exception as e:
        print(f"Error loading/cleaning data: {str(e)}")
        return None

sales_df = load_and_clean_data("Sales_Dataset.xlsx")

# %% [markdown]
"""
## Exploratory Data Analysis
"""
# %%
def perform_eda(df):
    """Perform exploratory data analysis on sales data"""
    
    # 1. Payment Method Analysis
    print("\n=== Payment Method Preferences ===")
    payment_dist = df['Payment_Method'].value_counts(normalize=True) * 100
    display(payment_dist.to_frame('Percentage'))
    
    plt.figure(figsize=(10, 5))
    sns.countplot(data=df, x='Payment_Method', order=payment_dist.index)
    plt.title('Payment Method Distribution')
    plt.xlabel('Payment Method')
    plt.ylabel('Count')
    plt.show()
    
    # 2. Product Performance Analysis
    print("\n=== Product Performance ===")
    
    # By Quantity
    product_quantity = df.groupby('Product')['Quantity'].sum().sort_values(ascending=False)
    print("\nTop Selling Products by Quantity:")
    display(product_quantity.head())
    
    plt.figure(figsize=(12, 6))
    product_quantity.plot(kind='bar', color='skyblue')
    plt.title('Total Quantity Sold by Product')
    plt.xlabel('Product')
    plt.ylabel('Quantity Sold')
    plt.xticks(rotation=45)
    plt.show()
    
    # By Revenue
    product_revenue = df.groupby('Product')['Revenue'].sum().sort_values(ascending=False)
    print("\nTop Selling Products by Revenue:")
    display(product_revenue.head())
    
    plt.figure(figsize=(12, 6))
    product_revenue.plot(kind='bar', color='lightgreen')
    plt.title('Total Revenue by Product')
    plt.xlabel('Product')
    plt.ylabel('Revenue ($)')
    plt.xticks(rotation=45)
    plt.show()
    
    # 3. Geographic and Manager Performance
    print("\n=== Performance by City and Manager ===")
    
    city_revenue = df.groupby('City')['Revenue'].sum().sort_values(ascending=False)
    print("\nTop Performing Cities by Revenue:")
    display(city_revenue.head())
    
    manager_revenue = df.groupby('Manager')['Revenue'].sum().sort_values(ascending=False)
    print("\nTop Performing Managers by Revenue:")
    display(manager_revenue.head())
    
    # 4. Time Series Analysis
    print("\n=== Revenue Trends Over Time ===")
    
    # Daily Revenue
    daily_revenue = df.groupby('Date')['Revenue'].sum()
    
    plt.figure(figsize=(14, 6))
    daily_revenue.plot(color='royalblue')
    plt.title('Daily Revenue Trends')
    plt.xlabel('Date')
    plt.ylabel('Revenue ($)')
    plt.grid(True)
    plt.show()
    
    # Monthly Revenue Comparison
    monthly_revenue = df.groupby('Month')['Revenue'].sum()
    print("\nMonthly Revenue Comparison:")
    display(monthly_revenue)
    
    # 5. Statistical Analysis
    print("\n=== Key Statistics ===")
    stats = df[['Quantity', 'Revenue']].agg(['mean', 'std', 'var']).transpose()
    display(stats)
    
    # 6. Product-wise Metrics
    print("\n=== Product Performance Metrics ===")
    product_metrics = df.groupby('Product').agg({
        'Quantity': 'mean',
        'Revenue': 'mean'
    }).sort_values('Revenue', ascending=False)
    display(product_metrics)
    
    return df

sales_df = perform_eda(sales_df)

# %% [markdown]
"""
## Advanced Analysis
"""
# %%
def advanced_analyses(df):
    """Perform more sophisticated analyses"""
    
    # Revenue growth rate
    monthly_revenue = df.groupby('Month')['Revenue'].sum()
    growth_rate = (monthly_revenue[12] - monthly_revenue[11]) / monthly_revenue[11] * 100
    print(f"\nMonth-over-Month Revenue Growth: {growth_rate:.2f}%")
    
    # Price elasticity analysis
    product_price_quantity = df.groupby('Product').agg({
        'Price': 'mean',
        'Quantity': 'sum'
    }).sort_values('Quantity', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=product_price_quantity, x='Price', y='Quantity', hue=product_price_quantity.index)
    plt.title('Price vs Quantity Sold')
    plt.xlabel('Average Price ($)')
    plt.ylabel('Total Quantity Sold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()
    
    # Payment method by city
    payment_by_city = pd.crosstab(df['City'], df['Payment_Method'], normalize='index') * 100
    print("\nPayment Method Distribution by City:")
    display(payment_by_city.style.background_gradient(cmap='Blues'))
    
    return df

sales_df = advanced_analyses(sales_df)

# %% [markdown]
"""
## Key Findings Summary
1. **Payment Preferences**:
   - Most popular method: [METHOD1] at [X]%
   - Least used method: [METHOD2] at [Y]%

2. **Product Performance**:
   - Top by quantity: [PRODUCT1] with [A] units
   - Top by revenue: [PRODUCT2] generating $[B]

3. **Geographic Insights**:
   - Highest revenue city: [CITY1] with $[C]
   - Lowest revenue city: [CITY2] with $[D]

4. **Time Trends**:
   - December revenue was [E]% higher than November
   - Peak sales day: [DATE] with $[F]

5. **Manager Performance**:
   - Top manager: [MANAGER1] with $[G]
   - Performance range: $[H] to $[I]

6. **Statistical Insights**:
   - Average order value: $[J]
   - Revenue variance: [K]
"""

# %% [markdown]
"""
## Recommendations
1. **Product Strategy**:
   - Focus promotions on [high-revenue products]
   - Review pricing for [low-quantity products]

2. **Payment Processing**:
   - Increase support for [popular payment method]
   - Consider incentives for [less popular methods]

3. **Staff Management**:
   - Recognize top-performing managers
   - Share best practices from [top city]

4. **Seasonal Planning**:
   - Prepare inventory for December surge
   - Analyze November underperformance
"""